In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, classification_report

In [2]:
# Read in the data
df = pd.read_csv(Path('merged_air_dia.csv'))
# drop rows with null values
df = df.dropna()

# print out the dataframe
df

,city,pm25,pm10,population,color_pm10,color_pm25,uniquezip,state,data_value_type,data_value,low_confidence_limit,high_confidence_limit
0,Anchorage,5.650000,11.781883,380821,green,green,0203000-02020002501,Alaska,Crude prevalence,7.4,6.9,7.8
1,Tempe,7.350000,15.326874,161780,green,green,0473000-04013319907,Arizona,Crude prevalence,7.0,6.4,7.9
2,Yuma,8.300000,17.307898,195751,green,green,0485540,Arizona,Crude prevalence,11.1,11.0,11.3
3,Folsom,6.250000,13.033056,72199,green,green,0624638,California,Crude prevalence,7.1,7.0,7.3
4,San Francisco,7.500000,15.639667,870887,green,green,0667000,California,Crude prevalence,8.6,8.5,8.6
...,...,...,...,...,...,...,...,...,...,...,...,...
12249,Charleston,7.300000,15.222609,227078,green,green,5414600-54039002000,West Virginia,Crude prevalence,10.3,9.3,11.3
12250,Yakima,8.550000,17.829220,243231,green,green,5380010-53077002802,Washington,Crude prevalence,9.0,8.4,9.5
12251,Yakima,8.550000,17.829220,243231,green,green,5380010-53077001000,Washington,Crude prevalence,8.9,8.4,9.4
12252,Tacoma,7.066667,14.736042,198397,green,green,5370000-53053061400,Washington,Crude prevalence,13.0,12.5,13.6


In [3]:
# create data frame for the columns that we do not need to scale or convert
unchanged_df = df[['pm25','pm10','population','data_value']]
unchanged_df.head()

,pm25,pm10,population,data_value
0,5.65,11.781883,380821,7.4
1,7.35,15.326874,161780,7.0
2,8.30,17.307898,195751,11.1
3,6.25,13.033056,72199,7.1
4,7.50,15.639667,870887,8.6


In [4]:
# check types for each column
df.dtypes

city                      object
pm25                     float64
pm10                     float64
population                 int64
color_pm10                object
color_pm25                object
uniquezip                 object
state                     object
data_value_type           object
data_value               float64
low_confidence_limit     float64
high_confidence_limit    float64
dtype: object

In [5]:
# df['color_pm25'].nunique() # 2 unique colors
# df['color_pm10'].nunique() # 3 unique colors
# df['data_value_type'].nunique() # 2 unique types

# drop high and low confidence limits for machine learning model so there is no bias
df.drop(columns=['low_confidence_limit','high_confidence_limit'],inplace=True)
df.head()

,city,pm25,pm10,population,color_pm10,color_pm25,uniquezip,state,data_value_type,data_value
0,Anchorage,5.65,11.781883,380821,green,green,0203000-02020002501,Alaska,Crude prevalence,7.4
1,Tempe,7.35,15.326874,161780,green,green,0473000-04013319907,Arizona,Crude prevalence,7.0
2,Yuma,8.30,17.307898,195751,green,green,0485540,Arizona,Crude prevalence,11.1
3,Folsom,6.25,13.033056,72199,green,green,0624638,California,Crude prevalence,7.1
4,San Francisco,7.50,15.639667,870887,green,green,0667000,California,Crude prevalence,8.6


In [6]:
# Convert columns that are objects to integers using onehotencoder
enc = OneHotEncoder(sparse=False)

# Fit and transform the onehotencoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[['color_pm25','color_pm10','data_value_type']]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names(['color_pm25','color_pm10','data_value_type'])
encode_df.head()

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0


In [7]:
# Use pandas.get_dummies to convert columns with a large amount of unique entries

# dummies_df = pd.get_dummies(df[['city','uniquezip','state']], columns=['city', 'uniquezip', 'state'])
dummies_df = pd.get_dummies(df[['uniquezip']], columns=['uniquezip'])
dummies_df.head()

,uniquezip_0203000,uniquezip_0203000-02020000101,uniquezip_0203000-02020000102,uniquezip_0203000-02020000201,uniquezip_0203000-02020000202,uniquezip_0203000-02020000203,uniquezip_0203000-02020000204,uniquezip_0203000-02020000300,uniquezip_0203000-02020000400,uniquezip_0203000-02020000500,...,uniquezip_5414600-54039001700,uniquezip_5414600-54039001800,uniquezip_5414600-54039001901,uniquezip_5414600-54039001902,uniquezip_5414600-54039002000,uniquezip_5414600-54039002100,uniquezip_5414600-54039011000,uniquezip_5414600-54039011401,uniquezip_5414600-54039012300,uniquezip_5414600-54039012800
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
# Merge encoded dataframes with the remaining columns
ml_df = pd.concat([encode_df, unchanged_df],axis=1) # took out dummies_df
ml_df.head()

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence,pm25,pm10,population,data_value
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,5.65,11.781883,380821,7.4
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.35,15.326874,161780,7.0
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.30,17.307898,195751,11.1
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,6.25,13.033056,72199,7.1
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.50,15.639667,870887,8.6


In [9]:
# check to make sure all of the data types are correct for ML
ml_df.dtypes

color_pm25_green                           float64
color_pm25_yellow                          float64
color_pm10_darkred                         float64
color_pm10_green                           float64
color_pm10_yellow                          float64
data_value_type_Age-adjusted prevalence    float64
data_value_type_Crude prevalence           float64
pm25                                       float64
pm10                                       float64
population                                   int64
data_value                                 float64
dtype: object

In [10]:
# Use standard scaler to help scale the data to train the model
scaler = StandardScaler()

# Want to scale the columns from data that are originally integers
cols_to_scale = ['pm10','pm25','population']
scaled_data = scaler.fit_transform(ml_df[cols_to_scale])
scaled_df = pd.DataFrame(scaled_data, columns=cols_to_scale)
scaled_df

,pm10,pm25,population
0,-1.280024,-1.280024,-0.823852
1,-0.396255,-0.396255,-0.899720
2,0.097616,0.097616,-0.887954
3,-0.968105,-0.968105,-0.930747
4,-0.318275,-0.318275,-0.654112
...,...,...,...
12249,-0.422248,-0.422248,-0.877103
12250,0.227582,0.227582,-0.871508
12251,0.227582,0.227582,-0.871508
12252,-0.543550,-0.543550,-0.887037


In [11]:
# drop the original columns that we scaled and add in the scaled columns to our ml_df
ml_df.drop(columns=cols_to_scale,inplace=True)
ml_df = pd.concat([ml_df, scaled_df],axis=1)
ml_df

,color_pm25_green,color_pm25_yellow,color_pm10_darkred,color_pm10_green,color_pm10_yellow,data_value_type_Age-adjusted prevalence,data_value_type_Crude prevalence,data_value,pm10,pm25,population
0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.4,-1.280024,-1.280024,-0.823852
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.0,-0.396255,-0.396255,-0.899720
2,1.0,0.0,0.0,1.0,0.0,0.0,1.0,11.1,0.097616,0.097616,-0.887954
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,7.1,-0.968105,-0.968105,-0.930747
4,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.6,-0.318275,-0.318275,-0.654112
...,...,...,...,...,...,...,...,...,...,...,...
12249,1.0,0.0,0.0,1.0,0.0,0.0,1.0,10.3,-0.422248,-0.422248,-0.877103
12250,1.0,0.0,0.0,1.0,0.0,0.0,1.0,9.0,0.227582,0.227582,-0.871508
12251,1.0,0.0,0.0,1.0,0.0,0.0,1.0,8.9,0.227582,0.227582,-0.871508
12252,1.0,0.0,0.0,1.0,0.0,0.0,1.0,13.0,-0.543550,-0.543550,-0.887037


In [12]:
# Split preprocessed data into features and target arrays
y = ml_df['data_value'].values
X = ml_df.drop(['data_value'],1).values

/Users/Kelly/opt/anaconda3/envs/mlenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [13]:
# check to see the shape of X
X.shape

(12254, 10)

In [14]:
y.shape # this is one column so that is what we want

(12254,)

In [15]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [16]:
# Create an object for linear regression
model = LinearRegression()

# time how long it takes to fit the model
start = datetime.now()

# Fit the linear regression model to the training set
model.fit(X_train,y_train)
end = datetime.now()

# print total time to fit and the RAM and CPU for machine
time_to_fit = end - start
print('Time to fit the model:',time_to_fit)
print('This code was run on a computer with memory: 4 GB 1600 MHz DDR3 and processor: 1.6 GHz Dual-Core Intel Core i5.')

Time to fit the model: 0:00:00.023708
This code was run on a computer with memory: 4 GB 1600 MHz DDR3 and processor: 1.6 GHz Dual-Core Intel Core i5.


In [17]:
# Predict the test set results
y_pred= model.predict(X_test)

In [20]:
# print summary stats to get r squared score
from statsmodels.api import OLS
OLS(y_test, X_test).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.069
Model:                            OLS   Adj. R-squared:                  0.067
Method:                 Least Squares   F-statistic:                     25.98
Date:                Sat, 21 May 2022   Prob (F-statistic):           1.66e-34
Time:                        11:14:02   Log-Likelihood:                -6944.6
No. Observations:                2451   AIC:                         1.391e+04
Df Residuals:                    2443   BIC:                         1.395e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1         -3.312e+10   1.64e+10     -2.026      0.043   -6.52e+10   -1.06e+09
x2         -3.312e+10   1.64e+10     -2.026      0.043   -6.52e+10   -1.06e+09
x3           1.25e+09   6.17e+08      2.026      0.043    3.99e+07    2.46e+09
x4           1.25e+09   6.17e+08      2.026      0.043    3.99e+07    2.46e+09
x5           1.25e+09   6.17e+08      2.026      0.043    3.99e+07    2.46e+09
x6          3.187e+10   1.57e+10      2.026      0.043    1.02e+09    6.27e+10
x7          3.187e+10   1.57e+10      2.026      0.043    1.02e+09    6.27e+10
x8         -2.619e+13   1.29e+13     -2.026      0.043   -5.16e+13   -8.37e+11
x9          2.619e+13   1.29e+13      2.026      0.043    8.37e+11    5.16e+13
x10            0.6002      0.090      6.655      0.000       0.423       0.777
==============================================================================
Omnibus:                      154.594   Durbin-Watson:                   1.932
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              186.485
Skew:                           0.624   Prob(JB):                     3.20e-41
Kurtosis:                       3.517   Cond. No.                     1.64e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.55e-29. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

In [21]:
# get more summary stats to see mean squared error
import sklearn.metrics as metrics
def regression_results(y_test, y_pred):

    # Regression metrics
    explained_variance=metrics.explained_variance_score(y_test, y_pred)
    mean_absolute_error=metrics.mean_absolute_error(y_test, y_pred) 
    mse=metrics.mean_squared_error(y_test, y_pred) 
    mean_squared_log_error=metrics.mean_squared_log_error(y_test, y_pred)
    median_absolute_error=metrics.median_absolute_error(y_test, y_pred)
    r2=metrics.r2_score(y_test, y_pred)

    print('explained_variance: ', round(explained_variance,4))    
    print('mean_squared_log_error: ', round(mean_squared_log_error,4))
    print('r2: ', round(r2,4))
    print('MAE: ', round(mean_absolute_error,4))
    print('MSE: ', round(mse,4))
    print('RMSE: ', round(np.sqrt(mse),4))

regression_results(y_test,y_pred)

explained_variance:  0.0685
mean_squared_log_error:  0.1305
r2:  0.0684
MAE:  3.1878
MSE:  16.9417
RMSE:  4.116
